# Melodic Expectation with Markov Models

In this notebook we will look at Markov Chains for modeling musical expectation.
We have already seen a Markov Model in the class on key estimation with HMMs (Hidden Markov Models).

## Information theoretic measures of musical expectation

* Information Content: Unexpectedness of a musical event

$$\text{IC}(\mathbf{x}_n\mid \mathbf{x}_{n-1}, \dots) = -\log_2 p(\mathbf{x}_n \mid \mathbf{x}_{n-1}, \dots )$$

* Entropy: How uncertain is a musical event

$$\text{H}(\mathbf{x}_{n-1:1})= \sum_{i \in \mathbf{A}} p(\mathbf{x}_i \mid \mathbf{x}_{n-1}, \dots) \text{IC}(\mathbf{x}_i\mid \mathbf{x}_{n-1}, \dots)$$

where $\mathbf{A}$ is the set of possible states that $\mathbf{x}$ can take.

## First order Markov Chains

The conditional distribution of each variable is independent of all previous observations except for the most recent

<div>
<img src="img/first_order_markov.png" width="250"/>
</div>

$$p(\mathbf{x}_1, \dots, \mathbf{x}_N) = p(\mathbf{x}_1)\prod_{n=2}^{N}p(\mathbf{x}_n \mid \mathbf{x}_{n-1})$$

The information content



In [ ]:
import os
# Uncomment this line if the kernel keeps crashing
# See https://stackoverflow.com/a/53014308
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import numpy as np
import partitura as pt
import matplotlib.pyplot as plt

%config InlineBackend.figure_format ='retina'

In [ ]:
from rnn import load_data

# To filter out short melodies by the minimum number of notes that a sequence should have

min_seq_len = 10
sequences = load_data(min_seq_len)

## Tasks 1; data loading & preparing:
1. check out the content of the variable "sequences", if unclear have a look at the loading function.
2. which musical texture do these sequences exhibit? (https://en.wikipedia.org/wiki/Texture_(music))
3. write a function to derive sequences of pitches from this data.
4. write a function to derive sequences of durations from this data. Modify this to compute inter onset intervals (IOIs; the time between two consecutive onsets). Can you encode rests as well by comparing duration with IOI? 

In [ ]:
features = list()
for seq in sequences:
    # pitch
    # features.append(seq["pitch"])
    # pitch class
    features.append(np.mod(seq["pitch"], 12))

In [ ]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
enc.fit(np.hstack(features))
encoded_sequences = [enc.transform(seq) for seq in features]

## Tasks 2; data exploration:

1. compute and draw a histogram of pitches. Modify this to show pitch classes!
2. compute and draw a histogram of IOIs. The input MIDI files are deadpan, i.e. the IOIs in seconds correspond to the notated duration exactly. Look through the IOIs and make an educated guess for some smallest float time unit that could serve as integer smallest time division. Encode the IOIs as multiples of this smallest integer. Which multiples make musical sense?

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

alphabet_size = len(enc.classes_)
alphabet = enc.classes_
print(f"Number of sequences: {len(sequences)}")
print(f"Unique elements {alphabet_size}")

fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(np.hstack(encoded_sequences), bins=alphabet_size, range=(0,alphabet_size), color='firebrick')
ax.set_xticks(np.arange(alphabet_size) + 0.5)
ax.set_xticklabels(alphabet)
ax.set_ylabel('Count')
plt.show()

## Tasks 3; A Markov Chain:

1. choose a data type to model: pitch, pitch class, IOIs, or durations (including or without an encoding for rests). Concatenate all the sequences into one long data sequence.

2. You have now a sequence **X** of symbols from an alphabet **A** (set of possible symbols of your chosen data type):

$$ \mathbf{X} = \{\mathbf{x_0}, \dots, \mathbf{x_n} \mid \mathbf{x}_{i} \in  \mathbf{A} \forall i \in 0, \dots, n \}$$

Compute the empirical conditional probability of seeing any symbol after just having seen any other:

$$ p(\mathbf{x_i}\mid \mathbf{x_{i-1}}) $$

What is the dimensionality of this probability  given $\lvert A \rvert = d $? Do you recall what this probability was called in the context of HMMs?


In [ ]:
def maximum_likelihood_fom(sequences, alphabet_size):
    probs = np.zeros((alphabet_size, alphabet_size)) + 1e-6
    for seq in sequences:
        for (p1, p2) in zip(seq[:-1],seq[1:]):
            probs[p1, p2] += 1
    probsum = np.sum(probs, axis = 1)
    normalized_distribution = (probs.T/probsum).T
    return normalized_distribution

transition_probabilities = maximum_likelihood_fom(encoded_sequences, alphabet_size)
fig, ax = plt.subplots(figsize=(7, 7))
a = ax.matshow(transition_probabilities, aspect='equal', cmap='BuPu')
ax.set_xticks(range(alphabet_size))
ax.set_xticklabels(enc.classes_)
ax.set_yticks(range(alphabet_size))
ax.set_yticklabels(enc.classes_)

plt.colorbar(a)
plt.show()

In [ ]:
def probability_of_sequence(sequence, transition_probs=transition_probabilities, alphabet_size=alphabet_size):
    """
    p(x_1,...,x_n)
    """
    p_0 = 1 / alphabet_size
    transitions = np.prod([transition_probs[i, j] for i, j in zip(sequence[1:],sequence[:-1])])
    prob = p_0 * transitions
    return prob

def information_content_fom(sequence, transition_probs=transition_probabilities, alphabet_size=alphabet_size):
    """
    For first order Markov models
    
    IC(x_n|x_{n-1},...,x_1) = IC(x_n|x_{n-1}) = - log2(p(x_n | x_{n-1}))
    """    
    ic = - np.log2(transition_probs[sequence[-1], sequence[-2]])
    return ic

def entropy_fom(sequence, alphabet=alphabet, transition_probs=transition_probabilities):
    """
    Entropy for first order Markov model
    """
    entropy_components = []
    for al in alphabet:
        in_seq = np.zeros_like(sequence)
        in_seq[:-1] = sequence[:-1]
        in_seq[-1] = al
        
        ic = information_content_fom(in_seq, transition_probs, alphabet_size=len(alphabet))
        prob = transition_probs[al, sequence[-2]]
        
        entropy_components.append(ic * prob)
    entropy = np.sum(entropy_components)
    
    return entropy


## Tasks 4; Markov Chain Generation:

1. By computing the probability $ \mathbb{P}(\mathbf{x_i}\mid \mathbf{x_{i-1}}) $ in task 3 you have fully specified a discrete-time finite state space Markov Chain model (https://en.wikipedia.org/wiki/Discrete-time_Markov_chain)! Given an initial symbol "s_0", you can generate the subsequent symbols by sampling from the conditional probability distribution

$$ p(\mathbf{x_i}\mid \mathbf{x_{i-1}} = \mathbf{s_{0}}) $$

Write a function that samples from a finite state space given an input probability distribution.

2. Use the previously defined function and the Markov Chain to write a sequence generator based on an initial symbol.
3. Start several "walkers", i.e. sampled/generated sequences. 


In [ ]:
from partitura.utils.synth import synthesize, SAMPLE_RATE
import IPython.display as ipd

def sample(distribution):
    cs = distribution.cumsum()
    samp = np.random.rand(1)
    return list(samp < cs).index(True)
    

def generate(start = 0, length = 100):
    melody = [start]
    for k in range(length - 1):
        melody.append(sample(transition_probabilities[melody[-1],:]))
    return np.array(melody)



def synthesize_generated_sequence(generated_sequence):
    note_array = np.zeros(
        len(generated_sequence),
        dtype=[
            ('onset_sec', 'f4'),
            ('duration_sec', 'f4'),
            ('pitch', 'i4'),
            ('velocity', 'i4'),
            ('id', 'U256')
        ]
    )
    note_array['pitch'] = generated_sequence + 60
    note_array['onset_sec'] = np.arange(len(generated_sequence)) * 0.5
    note_array['duration_sec'] += 0.5
    note_array['velocity'] = 64
    note_array['id'] = np.array([f'n{i}' for i in range(len(generated_sequence))])

    signal = synthesize(note_array, harmonic_dist='shepard')
    ipd.display(
            ipd.Audio(
                data=signal, 
                rate=SAMPLE_RATE, 
                normalize=False
            )
    )
    
    return note_array
    
n_notes = 100
generated_sequence = generate(length=n_notes)
print(generated_sequence)
note_array = synthesize_generated_sequence(generated_sequence)

## Tasks 5; n-gram Context Model:

1. The Markov Chains used until now have only very limited memory. In fact, they only ever know the last played pitch or duration. Longer memory models can be created by using the conditional probability of any new symbol based on an n-gram context of the symbol (https://en.wikipedia.org/wiki/N-gram):
$$ p(\mathbf{x_i}\mid \mathbf{x_{i-1}}, \dots, \mathbf{x_{i-n}}) $$

This probability will generally not look like a matrix anymore, but we can easily encode it as a dictionary. Write a function that creates a 3-gram context model from the data sequence **X**!

2. The longer the context, the more data we need to get meaningful or even existing samples for all contexts (note that the number of different contexts grows exponentially with context length). What could we do to approximate the distribution for unseen contexts?

In [ ]:
from collections import defaultdict
import copy

def create_context_model(sequences, n, n_states=alphabet_size):
    a_priori_probability = np.ones(n_states)/n_states
    context_model = defaultdict(lambda: copy.copy(a_priori_probability))
    for sequence in sequences:
        for idx in range(len(sequence)-n):
            local_string = ""
            for p in sequence[idx:idx+n]:
                local_string += str(p)
            context_model[local_string][sequence[idx+n]] += 1
    
    for key in context_model.keys():
        prob_dist = context_model[key]
        context_model[key] =  prob_dist/ prob_dist.sum()     
    
    return context_model
            
cm = create_context_model(encoded_sequences, 10)

In [ ]:
def generate_with_context_model(start = [0,0,0], length = 100, context_model= cm):
    melody = start
    for k in range(length):
        key = ""
        for p in melody[-len(start):]:
            key += str(p)
        melody.append(sample(context_model[key]))
    return np.array(melody)

generated_sequence = generate_with_context_model()

note_array = synthesize_generated_sequence(generated_sequence)

## Tasks 6; multi-type Markov Chains and back to music:

1. To generate a somewhat interesting melody, we want to get a sequence of both pitches and durations. If we encode rests too, we can generate any melody like this. So far our Markov Chains dealt with either pitch or duration/IOI. What could we do to combine them? Describe two approaches and why to choose which one.

2. Implement a simple melody generator with pitch and IOI/duration (simplest; modify taska 4; 2 to a generator of the other type and use them to create independent seuqnces). Write some generated melodies to MIDI files!

## (Tasks 7); more stuff for music:

1. Keys are perceptual centers of gravity in the pitch space, so if we transpose all the input sequences to the same key we can compute empirical pitch distributions within a key!

2. One solution to tasks 5, 2 is to use Prediction by Partial Matching. This is the basis of the most elaborate probabilitstic model ofsymbolic music the Information Dynamics of Music (IDyOM). See references here:
https://researchcommons.waikato.ac.nz/bitstream/handle/10289/9913/uow-cs-wp-1993-12.pdf
https://mtpearce.github.io/idyom/
